In [ ]:
!pip install google-generativeai langchain langchain-google-genai langgraph chromadb

!pip install -Uq "google-genai==1.7.0"

In [ ]:
from google import genai
from google.genai import types

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END
from typing import Dict, TypedDict
from langchain_core.messages import HumanMessage

from IPython.display import Markdown, display

genai.__version__
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
hahahahahah

## Testy 

In [ ]:

# Inicjalizacja modelu Gemini przez LangChain
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.3,
    google_api_key=GOOGLE_API_KEY
)


def explain_python_code(python_code: str) -> str:
    """
    Funkcja przyjmuje kod źródłowy w Pythonie i zwraca jego wyjaśnienie
    """
    prompt = f"""Jesteś pomocnym asystentem. Wyjaśnij krok po kroku, co robi ten kod w Pythonie:
        
    ```python
    {python_code}
    Wyjaśnij działanie każdej linijki. 
    """ 
    
    response = llm.invoke([HumanMessage(content=prompt)]) 
    return response.content

In [ ]:

sample_code = """ prompt = "Please provide me with 6 newest news From POLSANDODWAS" baseline_response = client.models.generate_content( model="gemini-1.5-flash-001", contents=[prompt]) print(baseline_response.text) """ 

explanation = explain_python_code(sample_code) 
print("\n🧠 Wyjaśnienie kodu:\n") 

print(explanation)

## Klasy definiujące agentów

In [ ]:
import google.generativeai as genai
from PIL import Image
import chromadb

# Ustawienie klucza API Gemini
genai.configure(api_key=GOOGLE_API_KEY)

for model in genai.list_models():
    print(f"Model: {model.name}")
    print(f"Supported methods: {model.supported_generation_methods}\n")


In [ ]:
# Inicjalizacja trwałego katalogu dla ChromaDB
import os

persistent_directory = "/kaggle/working/chromadb_storage"
os.makedirs(persistent_directory, exist_ok=True)


In [ ]:
import google.generativeai as genai
from PIL import Image
import chromadb

# Ustawienie klucza API Gemini
genai.configure(api_key=GOOGLE_API_KEY)

# Funkcja do generowania embeddingu zdjęcia
# Poprawiona funkcja do generowania embeddingu zdjęcia
def generate_image_embedding(image_path):
    image = Image.open(image_path)
    model = genai.GenerativeModel('models/gemini-1.5-flash')

    prompt = "Generate a descriptive caption for embedding purposes."
    response = model.generate_content([prompt, image])
    
    # Wygenerowanie embeddingu na podstawie tekstowego opisu zdjęcia
    embedding_model = genai.GenerativeModel('models/embedding-001')
    embedding = genai.embed_content(
        model='models/embedding-001',
        content=response.text,
        task_type="RETRIEVAL_DOCUMENT"
    )
    return embedding['embedding']
# Funkcja do automatycznego generowania opisu zdjęcia przez Gemini
def generate_image_caption(image_path):
    model = genai.GenerativeModel('models/gemini-1.5-flash')
    image = Image.open(image_path)

    prompt = "Provide a concise and accurate description of this image."
    response = model.generate_content([prompt, image])

    return response.text

# Inicjalizacja bazy ChromaDB
chroma_client = chromadb.PersistentClient(path=persistent_directory)
collection = chroma_client.get_or_create_collection(name="flag2")

# Funkcja do dodawania zdjęcia do bazy z automatycznym opisem
def add_image_to_db(image_path, image_id):
    embedding = generate_image_embedding(image_path)
    caption = generate_image_caption(image_path)

    collection.add(
        embeddings=[embedding],
        documents=[caption],
        ids=[image_id],
        metadatas=[{"path": image_path}]
    )

    print(f"Image '{image_id}' added to DB with caption: {caption}")

# Przykład użycia:
add_image_to_db("/kaggle/input/testgcc/IMG_1495.jpeg", "flag_pl")


In [ ]:
import matplotlib.pyplot as plt

# Funkcja do embeddingu zapytania tekstowego
def generate_query_embedding(query):
    embedding = genai.embed_content(
        model="models/embedding-001",
        content=query,
        task_type="RETRIEVAL_QUERY"
    )
    return embedding["embedding"]

# Agent wyszukujący najbliższy obraz na podstawie prompta
def find_image_by_prompt(prompt, collection, top_k=1):
    query_embedding = generate_query_embedding(prompt)
    
    # Wyszukiwanie w bazie
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    if not results['ids'][0]:
        print("Brak wyników.")
        return

    image_path = results['metadatas'][0][0]['path']
    caption = results['documents'][0][0]
    print(f"Najlepiej pasujący opis: {caption}")

    # Wyświetlanie obrazu
    image = Image.open(image_path)
    plt.imshow(image)
    plt.axis('off')
    plt.title(caption)
    plt.show()

# Przykład użycia:
find_image_by_prompt("flaga polski", collection)


### Code Explainer 

In [ ]:
# Create the CodeExplainer agent
explainer_agent = CodeExplainer(name="CodeExplainer", model="gemini-1.5-flash", temperature=0.2)

# We will now use the LLM as a "judge" to evaluate correctness or validity of the code

judge_prompt = f"""
You are a strict yet fair code judge. Evaluate the correctness of the agent's response in the context of the provided code snippet.

Consider the following criteria:
1. Logic and correctness: Is the solution logically sound and functionally correct?
2. Best practices: Does the agent adhere to good coding conventions and best practices?
3. Clarity: Is the explanation well-structured, understandable, and adequately justified?

Provide:
- A single overall score from 1 to 5 (1 = very poor, 5 = excellent).
- A concise explanation (1–2 sentences) summarizing the main reasons for your rating.

Code:
{sample_code}

Agent Response:
{explanation}
"""

 # We directly call the LLM client on our judge prompt 
judgement_response = explainer_agent.llm.invoke([HumanMessage(content=judge_prompt)]) 
print("\n=== LLM AS JUDGE ===\n") 
print(judgement_response.content)